In [25]:
import os
import json
import numpy as np
from typing import List, Dict
from tqdm import tqdm
# from embedding_model import get_embeddings as get_embedding
from embeddings_openai import embed_large as get_embedding
from voyage import rerank

# Directory containing the embedding files
db_dir = 'db_openai_large_8000'

# Function to load embedding data
def load_embedding_data(db_dir: str) -> List[Dict]:
    data = []
    for filename in os.listdir(db_dir):
        if filename.endswith('.json'):
            file_path = os.path.join(db_dir, filename)
            with open(file_path, 'r', encoding='utf-8') as f:
                data.extend(json.load(f))
    return data

# Load the data
embedding_data = load_embedding_data(db_dir)
print(f"Loaded {len(embedding_data)} entries.")

Loaded 1989 entries.


In [26]:

# Extract texts and embeddings from the data
texts = []
embeddings = []

for i in range(len(embedding_data)):
    texts.append(embedding_data[i]['name']['text'])
    embeddings.append(embedding_data[i]['introduction']['embedding'])
    for value in embedding_data[i]['chunks'].values():
        texts.append(value['text'])
        embeddings.append(value['embedding'])



In [27]:
from IPython.display import display, HTML

def search_query(query: str):
    # query_embedding = get_embedding([query])[0]
    query_embedding = get_embedding(query)
    scores = np.dot(embeddings, query_embedding)
    top_indices = np.argsort(scores)[-10:][::-1]
    top_results = [{'score': scores[i], 'text': texts[i]} for i in top_indices]
    
    formatted_results = []
    for result in top_results:
        for entry in embedding_data:
            for key, value in entry.items():
                try:
                    if value['text'] == result['text']:
                        formatted_results.append({
                            'score': result['score'],
                            'name': entry.get('name', {}).get('text', ''),
                            'introduction': entry.get('introduction', {}).get('text', ''),
                            'text': result['text']
                        })
                except KeyError:
                    for chunk in value.values():
                        if chunk['text'] == result['text']:
                            formatted_results.append({
                                'score': result['score'],
                                'name': entry.get('name', {}).get('text', ''),
                                'introduction': entry.get('introduction', {}).get('text', ''),
                                'text': result['text']
                            })

    formatted_results = sorted(formatted_results, key=lambda x: x['score'], reverse=True)
    return formatted_results



In [28]:
# Easy
query1 = "Ve kterém zákoně a v jakém paragrafu je upraveno parazitování na pověsti?"
query2 = "Jaký trest hrozí za loupež?"
query3 = "Kolik hodin ročně lze maximálně odpracovat na dohodu o provedení práce?"
query4 = "Jaká je lhůta pro podání odporu v trestním řízení?"
query5 = "Vysvětli mi, co je to nekalá soutěž"
query6 = "Jaká je promlčecí doba dle občanského zákoníku?"
query7 = "Kolik je pro rok 2024 existenční minimum?"

# Medium
query8 = "Jaké podmínky musím splnit pro získání živnostenského oprávnění optometristy"
query9 = "Za splnění jakých podmínek nemusím platit daň z příjmu jakožto fyzická osoba při prodeji nemovité věci? Nemovitou věc jsem si zakoupil v dubnu 2022."
query = query9
results = search_query(query)

processed_results = []
for result in results[:30]:
    processed_results.append("Název zákonu: " + result['name'] + '\n\n' + "Znění zákonu: " + result['text'])

# Display the results
for result in results[:10]:
    display(HTML(f"<b>Score:</b> {result['score']}<br><b>Name:</b> {result['name']}<br><b>Introduction:</b> {result['introduction']}<br><b>Text:</b> {result['text']}<br><br>"))


reranked_results = rerank(query, processed_results)
for reranked_result in reranked_results:
  # print(reranked_result.relevance_score)
  print(reranked_result.document, '\n')

Název zákonu: Zákon o daních z příjmů

Znění zákonu: zh)
zi) příjmy podle § 4a.


zi)
(2) Doba mezi nabytím a prodejem podle odstavce 1 písm. a) nebo b) se nepřerušuje, pokud v době mezi nabytím a prodejem došlo k


(2)
a) vypořádání mezi spoluvlastníky nemovité věci rozdělením podle velikosti jejich podílů,


a)
b) tomu, že v domě vznikly jednotky,


b)
c) vypořádání společného jmění manželů nebo


c)
d) rozdělení pozemku.


d)
(4) Obchodním majetkem poplatníka daně z příjmů fyzických osob se pro účely daní z příjmů rozumí část majetku poplatníka, o které bylo nebo je účtováno anebo je nebo byla uvedena v daňové evidenci. Dnem vyřazení určité složky majetku z obchodního majetku poplatníka se rozumí den, kdy poplatník o této složce majetku naposledy účtoval nebo ji naposledy uváděl v daňové evidenci.


(4)
§ 4a


Osvobození bezúplatných příjmů

Od daně z příjmů fyzických osob se osvobozuje bezúplatný příjem


a) z nabytí dědictví nebo odkazu,


a)
b) obmyšleného z majetku, který


b)
1